### **Organize PSD Data**

- [ ]  Ensure all power spectral data (PSD) is saved per **subject-session** in a consistent format:
    - One file or object per subject-session
    - Contains:
        - EEG **channels**
        - **Frequencies**
        - Power per **epoch** or pre-averaged by condition
    - Associated **metadata**:
        - Subject ID
        - Session ID 
        - Dataset name
        - Cognitive **state label** (OT, MW)

In [1]:
from eeg_analyzer.dataset import Dataset
from utils.config import DATASETS

dataset_config = DATASETS['jin2019']

dataset = Dataset(dataset_config)
dataset.load_subjects()

### **Extract Alpha Power**

- [ ]  For each subject-session
    - Select **8–12 Hz** frequency band
    - **Sum power across frequencies** per channel and epoch
- [ ]  Result per epoch:

```python
{
    "subject_session": "001_1",
    "subject_id": "001",
    "session_id": "1",
    "channel": "Pz",
    "task": "SART",
    "state": "MW",
    "alpha_power": 3.45,
    ...
}
```

### **Assign Numerical Condition Labels**

For modeling the *ordinal direction* of alpha power:

| State | Code |
| --- | --- |
| OT | 0 |
| MW | 1 |
| MED | 2 |
- [ ]  For **Jin** and **Touryan**, you only use codes `0` and `1` (OT, MW)
- [ ]  For **Braboszcz**, use codes `1` and `2` (MW, MED)

In [2]:
# Get all list of all epochs for all subject-session pairs as dict with these keys:
# 'subject_session', 'subject', 'session', 'channel', 'task', 'state', 'band_power'

epochs = dataset.to_long_band_power_list(
    freq_band=(8,12),  # Alpha band
    use_rois=False
)

estimated_length = dataset.estimate_long_band_power_length()

### **Create Long-Form DataFrame per Dataset**

For each dataset, prepare:
| subject_session | subject_id | group | channel/ROI | state | alpha_power |
| --- | --- | --- | --- | --- | --- |
| 001_1 | 001 | NaN | Pz | 0 | 3.24 |
| 001_1 | 001 | NaN | Pz | 1 | 3.88 |
| 060_1 | 060 | vip | Pz | 1 | 5.12 |
| 060_1 | 060 | vip | Pz | 2 | 6.00 |

In [3]:
# create a dataframe with the epochs
import pandas as pd
df_full = pd.DataFrame(epochs)

## **Make grouped DataFrames per channel**

* [ ] Create a list of DataFrames for each its own channel.
* [ ] Create a new column with the z-score within state.
* [ ] Filter the data by removing all epochs with a z-score > 4.

In [ ]:
# Create a list of dataframes for each channel (keep the channel column)
df_channels = []
for channel in df_full['channel'].unique():
    df_channel = df_full[df_full['channel'] == channel].reset_index(drop=True)
    df_channels.append(df_channel)

# create a new column with z-scores within each state
def z_score_within_state(df):
    grouped = df.groupby(['subject_session', 'state'])
    df['z_score'] = grouped['band_power'].transform(lambda x: (x - x.mean()) / x.std())
    return df

# apply the z-score function to each channel dataframe
df_channels_z = []
for df_channel in df_channels:
    df_channel_z = z_score_within_state(df_channel)
    df_channels_z.append(df_channel_z)

# Remove all rows where z_score > 3
for i in range(len(df_channels_z)):
    df_channels_z[i] = df_channels_z[i][df_channels_z[i]['z_score'] <= 3]

print(df_channels_z[0].head())


### **Fit Mixed Effects Model per Channel**

Do this **separately for each dataset** and for each channel (or ROI):

### ✅ Model formula:

```python
alpha_power ~ state + (state | subject_session)
```

This:

- Estimates the effect of `state` (e.g., OT → MW or MW → MED)
- Models per-subject variability in both baseline (intercept) and sensitivity (slope)

In [ ]:
# Fit mixed effects model per channel
import statsmodels.formula.api as smf

results_per_channel = {}

for df_channel_z in df_channels_z:
    channel_name = df_channel_z['channel'].iloc[0] if 'channel' in df_channel_z else None
    # Fit the mixed effects model: alpha_power ~ state + (state | subject_session)
    # Use 'band_power' as the dependent variable
    # 'state' as fixed effect, random intercept and slope for 'subject_session'
    model = smf.mixedlm(
        "band_power ~ state",
        df_channel_z,
        groups=df_channel_z["subject_session"],
        re_formula="~state"
    )
    result = model.fit()
    results_per_channel[channel_name] = result

# Example: print summary for one channel
first_channel = list(results_per_channel.keys())[0]
print(f"Results for channel: {first_channel}")
print(results_per_channel[first_channel].summary())

In [ ]:
# print summary for all channels
for channel_name, result in results_per_channel.items():
    print(f"Results for channel: {channel_name}")
    print(result.summary())
    print("\n")

### **Store & Summarize Results**

For each model/channel:

- [ ]  Store:
    - Fixed effect estimate for `state`
    - p-value
    - t-statistic (optional)
    - Number of subjects included
- [ ]  Save results to `.csv` or `.json`

### **Visualize Results**

- [ ]  Plot per-channel **bar plots** or **line plots** of alpha power by condition
- [ ]  Plot **topoplots** (scalp maps) of:
    - `state` slope per channel
    - p-values (FDR-corrected or raw, with masking)
    - t-values

In [ ]:
import matplotlib.pyplot as plt
from utils.config import PLOTS_PATH
import os



### **Correct for Multiple Comparisons**

- [ ]  Across channels (e.g., for scalp maps), correct p-values using:
    - **False Discovery Rate (FDR)**
    - Or **cluster-based permutation test** if possible

### **Filter Low-Effect Subjects**

- [ ]  Compute within-subject effect sizes (e.g., Cohen’s *d*) between states
- [ ]  Re-run the model excluding sessions with **d < 0.5**
- [ ]  Compare results to check robustness